This notebook uses Python to review the basics of $\chi^2$ tests. They are a way to test for significance for categorical variables.

I would also like to more completely understand the critical values used to compare $\chi^2$ values depending on the degrees of freedom.

In [3]:
#packages used in this notebook
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.style.use("ggplot")

In [19]:
#generate random numbers to be categories
def simple_categorical(size=10):
    '''
    Return a list of the given length of two pseudo-randomly-generated categories "first" and "second", relying on Python's
    random.randint method.
    '''
    cat_list=[]
    
    for i in range(size): #loop through number of times given by size argument
        num = random.randint(1,2)
        if num ==1:
            cat="first"
        elif num==2:
            cat="second"
        cat_list.append(cat)
    
    return cat_list #return the list of Strings pseudo-randomly chosen

In [20]:
firseclist = simple_categorical()

In [23]:
#create a dataframe with categorical variables
newframe = pd.DataFrame({"catvar1":firseclist},index=range(10))
newframe

,catvar1
0,second
1,first
2,first
3,second
4,second
5,second
6,second
7,first
8,second
9,first
